In [ ]:
"""
A script to plot the orientation auto-correlation function for a VASP molecular dynamics run. Accepts VASP XDATCAR file.
Requires pymatgen and matplotlib.
Run in calculation folder for a single run, or in folder containing run subfolders for multiple calc simulations.
"""
from pymatgen.io.vasp.outputs import Xdatcar
from pymatgen.core import structure
import numpy as np
import matplotlib.pyplot as plt
import os



"""
Input:
VASP XDATCAR

Required parameters:
overlap - amount of structures to shift next time origin by
primary species - central atom species in polyanion
secondary_species - secondary atom in polyanion
expected_s - number of secondary atoms in polyanion
runs - list of XDATCAR locations (will be concatenated)
plot_length - length of plot (fs)
step_skip - time difference between adjacent structures in XDATCAR (most likely POTIM * NBLOCK)

Output:
Plots OACF and saves PNG to current directory.
"""
overlap = 5
primary_species = 'P'
secondary_species = 'S'
expected_s = 4
runs = ['./']
step_skip = 50
plot_length = 100
plot_title = "Li3PS4 OACF"

In [ ]:
def periodics(vector): # dealing with PBCs (accounting for if an ion moves out of one side of the cell)
    tmp = []
    for element in vector:
        if abs(element) > 0.5:
            if element > 0.5:
                element-=1
            else:
                element+=1
        tmp.append(element)
    vector = tuple(tmp)
    return vector

def OACF(primary_species,secondary_species,runs,overlap,plot_length):
    # sticking all XDATCAR files for the sim into one object
    data = Xdatcar(f'{runs[0]}XDATCAR')
    if len(runs) > 1:
        for run in runs[1:]:
            data.concatenate(f"{run}XDATCAR")
    print(f"Number of structures = {len(data.structures)}")
    orig_struc  = data.structures[0] #marking the original locations of the PS4 polyanions
    p_indices = []
    for n,atom in enumerate(orig_struc): # Scanning structure for primary indices
        if primary_species in str(atom.specie):
            p_indices.append(n)

    struc_data = {}
    for p_site in p_indices: # Scanning structure for nearest secondary indices
        radius = 3
        s_indices = orig_struc.get_neighbors(orig_struc[p_site],r=radius)
        passes = 1
        while len(s_indices) != expected_s:
            if passes <=10:
                if len(s_indices) > 4:
                    radius -= 0.1
                elif len(s_indices) < 4:
                    radius += 0.1
            elif passes <=30:
                if len(s_indices) > 4:
                    radius -= 0.01
                elif len(s_indices) < 4:
                    radius += 0.01
            else:
                if len(s_indices) > 4:
                    radius -= 0.001
                elif len(s_indices) < 4:
                    radius += 0.001
            s_indices = orig_struc.get_neighbors(orig_struc[p_site],r=radius)
            passes += 1
        s_indices = [i.index for i in s_indices if secondary_species in str(i.specie)]
        struc_data[p_site] = s_indices

    OACF_data = {}
    final_struc = len(data.structures) - 1 # steps implementing multiple time origin stuff
    first_struc_no = 0

    last_struc_no = first_struc_no + plot_length

    struc_vectors = {}
    for frame in range(0, len(data.structures)): #
            current_struc = data.structures[frame]
            struc_vectors[frame] = {}
            for phos in struc_data.keys():
                struc_vectors[frame][phos] = {}
                for sulph in struc_data[phos]:
                    vector = current_struc[sulph].frac_coords - current_struc[phos].frac_coords
                    
                    vector = periodics(vector) / np.linalg.norm(periodics(vector))
                    struc_vectors[frame][phos][sulph] = vector

    while last_struc_no <= final_struc: #iterate through the chunks
        first_struc  = data.structures[first_struc_no]
        mean_OACFs = np.array([])
        for current_struc in range(first_struc_no,last_struc_no): #
            OACFs = np.array([])
            for phos in struc_data.keys():
                for sulph in struc_data[phos]:
                    old_vector = struc_vectors[first_struc_no][phos][sulph]
                    new_vector = struc_vectors[current_struc][phos][sulph]
                    
                    # calculating OACF
                    OACF = np.dot(np.array(old_vector),np.array(new_vector))
                    OACF = np.array([round(OACF,5)])
                    OACFs = np.append(OACFs,OACF)
            mean_OACF = np.mean(OACFs)
            mean_OACFs = np.append(mean_OACFs,mean_OACF)
        OACF_data[f'set_{first_struc_no}'] = mean_OACFs
        first_struc_no += overlap
        last_struc_no += overlap
    mean_mean_OACFs = np.array([])
    for i in range(len(OACF_data['set_0'])):
        current_means = np.array([])
        for j in OACF_data.keys():
            current_means = np.append(current_means,OACF_data[j][i])
        mean_current_means = np.mean(current_means)
        mean_mean_OACFs = np.append(mean_mean_OACFs,mean_current_means)
    frames = []

    #plotting
    for n,i in enumerate(mean_mean_OACFs):
        frames.append(n*step_skip)
    plt.plot(frames, mean_mean_OACFs)
    plt.title(plot_title)
    plt.xlabel("$\Delta$t / fs")
    plt.ylabel("Average OACF")
    plt.savefig(f"./OACF.png")
    plt.show()

In [ ]:
OACF(primary_species,secondary_species,runs,overlap,plot_length)